In [1]:
!pip install llama-index llama-index-readers-web duckduckgo_search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 839.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 25.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.9/37.9 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8

# Identify specific region(s) that a location belongs to (used to match location to corresponding data regulation policies)

### Define locations and link to regulations

In [4]:
import json
import pandas as pd

In [6]:
data = pd.read_csv('/content/Region Data Regulation.csv')

In [ ]:
with open('/content/region_data_regulations.json') as f:
  data = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/content/region_data_regulations.json'

In [7]:
data

,regulations,regions,links
0,General Data Protection Regulation (GDPR),European Union,https://eur-lex.europa.eu/legal-content/EN/TXT...
1,California Consumer Privacy Act (CCPA),California,https://leginfo.legislature.ca.gov/faces/codes...
2,Personal Information Protection and Electronic...,Canada,https://laws-lois.justice.gc.ca/ENG/ACTS/P-8.6...
3,Privacy Act 1988,Australia,https://www.legislation.gov.au/C2024A00024/lat...
4,Data Protection Act 2018,United Kingdom,https://www.legislation.gov.uk/ukpga/2018/12/c...
5,APPI (Act on the Protection of Personal Inform...,Japan,https://www.japaneselawtranslation.go.jp/en/la...
6,PIPA (Personal Information Protection Act),South Korea,https://www.law.go.kr/LSW/lsInfoP.do?lsiSeq=21...
7,LGPD (Lei Geral de Proteção de Dados),Brazil,https://www.planalto.gov.br/ccivil_03/_ato2015...
8,DPDP Act (Digital Personal Data Protection Act),India,https://meity.gov.in/writereaddata/files/Digit...
9,POPIA (Protection of Personal Information Act),South Africa,https://popia.co.za/


In [14]:
region_data_regulation = dict(zip(data['regions'], data['links']))

In [15]:
def get_data_regulation_url(region_data_regulation, region_name):
    return region_data_regulation.get(region_name, "Data regulation URL not found for this region")

### LLM (identify location)

In [16]:
import os

import regex as re
import pandas as pd
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core import SummaryIndex, VectorStoreIndex
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core.tools import BaseTool, FunctionTool
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.agent import ReActAgent
from IPython.display import Markdown, display
from duckduckgo_search import DDGS
os.environ["OPENAI_API_KEY"] = "sk-proj-e20Q6bKuRvBVQBObIFhvT3BlbkFJ3HdudAgJER9AnPxwsgFp"

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [17]:
token = 'sk-proj-e20Q6bKuRvBVQBObIFhvT3BlbkFJ3HdudAgJER9AnPxwsgFp'
llm = OpenAI(model="gpt-4o", openai_api_key=token)
Settings.llm = llm

In [18]:
REGION_TEMPLATE = """
    Assistant: I can help determine which region a user input belongs to. The options are: United States, California, European Union, Canada, Australia, United Kingdom, Japan, South Korea, South Africa, Singapore, Brazil, Other. Make sure to include all options.
    User: {user_input}
    Assistant: The regions that the user input belongs to is (are):
    Example:
    User: los angeles
    Assistant: California, United States.
    User: hong kong
    Assistant: Other.
    """

In [30]:
def determine_region(user_input, api_key = "sk-proj-e20Q6bKuRvBVQBObIFhvT3BlbkFJ3HdudAgJER9AnPxwsgFp", temperature=0.3):
    # Initialize the OpenAI model with llama-index
    llm = OpenAI(model="gpt-4o", openai_api_key=api_key)

    REGION_TEMPLATE = """
    Assistant: I can help determine which region a user input belongs to. The options are: California, European Union, Canada, Australia, United Kingdom, Japan, South Korea, South Africa, Singapore, Brazil, Other. Make sure to include all options.

    Examples:
    User: los angeles
    Assistant: California
    User: hong kong
    Assistant: Other
    User: London
    Assistant: European Union, United Kingdom
    User: toronto
    Assistant: Canada
    Now, let's try with your input.
    User: {user_input}
    Assistant: The regions that the user input belongs to are:
    """

    prompt = REGION_TEMPLATE.format(user_input=user_input)

    response = llm.complete(prompt=prompt, max_tokens=50, temperature=temperature)

    answer = response.text.strip().replace("The regions that the user input belongs to is (are):", "").strip()

    return answer

In [25]:
def clean_region_output(text):
  return text.split('\n')[0]

In [26]:
def get_privacy_law_links(user_input, api_key = "sk-proj-e20Q6bKuRvBVQBObIFhvT3BlbkFJ3HdudAgJER9AnPxwsgFp"):
    regions = determine_region(user_input, api_key)
    regions = clean_region_output(regions)
    region_list = regions.split(', ') # Assuming regions are comma-separated
    links = []
    for region in region_list:
        link = get_data_regulation_url(region_data_regulation, region.strip())
        links.append(link)
    return region_list, links

In [27]:
la_regions, la_links = get_privacy_law_links("los angeles")
la_links

['https://leginfo.legislature.ca.gov/faces/codes_displayText.xhtml?division=3.&part=4.&lawCode=CIV&title=1.81.5']

In [35]:
clean_region_output(determine_region("tokyo"))

'Japan'

In [36]:
tokyo_links = get_privacy_law_links("tokyo")
tokyo_links

(['Japan'], ['https://www.japaneselawtranslation.go.jp/en/laws/view/4241/en'])

In [37]:
beijing_links = get_privacy_law_links("beijing")
beijing_links

(['Other'], ['Data regulation URL not found for this region'])

In [38]:
london_links = get_privacy_law_links("london")
london_links

(['European Union', 'United Kingdom'],
 ['https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32016L0680',
  'https://www.legislation.gov.uk/ukpga/2018/12/contents'])

In [39]:
for i in range(10):
  print(get_privacy_law_links("los angeles"))

(['California'], ['https://leginfo.legislature.ca.gov/faces/codes_displayText.xhtml?division=3.&part=4.&lawCode=CIV&title=1.81.5'])
(['California'], ['https://leginfo.legislature.ca.gov/faces/codes_displayText.xhtml?division=3.&part=4.&lawCode=CIV&title=1.81.5'])
(['California'], ['https://leginfo.legislature.ca.gov/faces/codes_displayText.xhtml?division=3.&part=4.&lawCode=CIV&title=1.81.5'])
(['California'], ['https://leginfo.legislature.ca.gov/faces/codes_displayText.xhtml?division=3.&part=4.&lawCode=CIV&title=1.81.5'])
(['California'], ['https://leginfo.legislature.ca.gov/faces/codes_displayText.xhtml?division=3.&part=4.&lawCode=CIV&title=1.81.5'])
(['California'], ['https://leginfo.legislature.ca.gov/faces/codes_displayText.xhtml?division=3.&part=4.&lawCode=CIV&title=1.81.5'])
(['California'], ['https://leginfo.legislature.ca.gov/faces/codes_displayText.xhtml?division=3.&part=4.&lawCode=CIV&title=1.81.5'])
(['California'], ['https://leginfo.legislature.ca.gov/faces/codes_displayTex

In [ ]:
for i in range(10):
  print(get_privacy_law_links("san diego"))

California, United States
['https://cppa.ca.gov/regulations/pdf/cppa_act.pdf', 'https://www.ftc.gov/legal-library/browse/statutes/ftc-act']
California, United States
['https://cppa.ca.gov/regulations/pdf/cppa_act.pdf', 'https://www.ftc.gov/legal-library/browse/statutes/ftc-act']
California, United States
['https://cppa.ca.gov/regulations/pdf/cppa_act.pdf', 'https://www.ftc.gov/legal-library/browse/statutes/ftc-act']
California, United States
['https://cppa.ca.gov/regulations/pdf/cppa_act.pdf', 'https://www.ftc.gov/legal-library/browse/statutes/ftc-act']
California, United States
['https://cppa.ca.gov/regulations/pdf/cppa_act.pdf', 'https://www.ftc.gov/legal-library/browse/statutes/ftc-act']
California, United States
['https://cppa.ca.gov/regulations/pdf/cppa_act.pdf', 'https://www.ftc.gov/legal-library/browse/statutes/ftc-act']
California, United States
['https://cppa.ca.gov/regulations/pdf/cppa_act.pdf', 'https://www.ftc.gov/legal-library/browse/statutes/ftc-act']
California, United S

In [ ]:
la_reader = SimpleWebPageReader(html_to_text=True).load_data(la_links)

In [ ]:
la_data_regulation_query_tool = [
    QueryEngineTool(
        query_engine=VectorStoreIndex.from_documents([la_reader[i]]).as_query_engine(),
        metadata=ToolMetadata(
            name=f"data_regulation_for_{la_regions[i]}",
            description=("useful for when you want to know "
                         f"{la_regions[i]}'s data regulation"),
        ),
    ) for i in range(len(la_links))
]

In [ ]:
la_data_regulation_query_tool[0].query_engine.query('What is the data protection law for California about?').response

'The data protection law for California, known as the California Consumer Privacy Act of 2018, focuses on providing privacy rights and consumer protection for residents of California. It grants consumers the right to know what personal data is being collected about them, to whom it is being sold or disclosed, and the ability to access, delete, and opt-out of the sale of their personal information.'

In [ ]:
la_data_regulation_query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=la_data_regulation_query_tool)

In [ ]:
la_data_regulation_query_engine_tool = QueryEngineTool(
    query_engine=la_data_regulation_query_engine,
    metadata=ToolMetadata(
        name="sub_question_query_engine_for_data_regulation",
        description=(
            "useful for when you want to answer queries that require analyzing"
            f" multiple data regulations from different regions, including: {', '.join(la_regions)}"),
    ),
)

In [ ]:
la_data_regulation_query_engine_tool.query_engine.query('What are the common elements of the data protection laws for los angeles?').response

Generated 2 sub questions.
[data_regulation_for_California] Q: What are the data protection laws specific to California?
[data_regulation_for_United States] Q: What are the data protection laws specific to the United States?
[data_regulation_for_California] A: The data protection law specific to California is the California Consumer Privacy Act of 2018.
[data_regulation_for_United States] A: To find information on data protection laws specific to the United States, you can search or browse the Legal Library on the relevant government website. This resource provides legal guidance and documents to help understand business responsibilities and compliance with the law.


'Common elements of data protection laws for Los Angeles would include compliance with the California Consumer Privacy Act of 2018, which is a state-wide regulation. This act focuses on consumer rights regarding personal data, including the right to know what data is being collected, the right to delete personal data, and the right to opt-out of the sale of personal data. Additionally, businesses must provide transparency about their data collection practices and ensure the protection of consumer data.'

In [ ]:
def get_data_regulation_info(location, api_key="sk-proj-e20Q6bKuRvBVQBObIFhvT3BlbkFJ3HdudAgJER9AnPxwsgFp"):
    """
    This function takes a location as input and returns a query engine tool
    that can be used to answer questions about data regulations in that location.
    """
    regions, links = get_privacy_law_links(location, api_key)
    if 'Privacy law URL not found for this region' in links:
        print('Data regulation law URL not found for this location.')
        return None
    reader = SimpleWebPageReader(html_to_text=True).load_data(links)
    data_regulation_query_tool = [
        QueryEngineTool(
            query_engine=VectorStoreIndex.from_documents([reader[i]]).as_query_engine(),
            metadata=ToolMetadata(
                name=f"data_regulation_for_{regions[i]}",
                description=("useful for when you want to know "
                             f"{regions[i]}'s data regulation"),
            ),
        ) for i in range(len(links))
    ]
    data_regulation_query_engine = SubQuestionQueryEngine.from_defaults(
        query_engine_tools=data_regulation_query_tool)
    data_regulation_query_engine_tool = QueryEngineTool(
        query_engine=data_regulation_query_engine,
        metadata=ToolMetadata(
            name="sub_question_query_engine_for_data_regulation",
            description=(
                "useful for when you want to answer queries that require analyzing"
                f" multiple data regulations from different regions, including: {', '.join(regions)}"),
        ),
    )
    return data_regulation_query_engine_tool


In [ ]:
location = "los angeles"
query_engine_tool = get_data_regulation_info(location)
response = query_engine_tool.query_engine.query('What are the common elements of the data protection laws for los angeles?').response
print(response)

Generated 2 sub questions.
[data_regulation_for_California] Q: What are the data protection laws specific to California?
[data_regulation_for_United States] Q: What are the data protection laws specific to the United States?
[data_regulation_for_United States] A: The United States has several federal laws that address data protection and privacy, including the Health Insurance Portability and Accountability Act (HIPAA) for health information, the Gramm-Leach-Bliley Act (GLBA) for financial information, and the Children's Online Privacy Protection Act (COPPA) for children's information. Additionally, the Federal Trade Commission (FTC) enforces laws that protect consumers from unfair or deceptive practices, which can include data protection and privacy issues.
[data_regulation_for_California] A: The data protection law specific to California is the California Consumer Privacy Act of 2018.
Common elements of data protection laws in Los Angeles would include provisions from the California 

In [ ]:
location = "tokyo"
query_engine_tool = get_data_regulation_info(location)
response = query_engine_tool.query_engine.query(f'What are the common elements of the data protection laws for {location}?').response
print(response)

Generated 5 sub questions.
[data_regulation_for_Japan] Q: What are the key principles of data protection laws in Tokyo?
[data_regulation_for_Japan] Q: What are the requirements for data processing under Tokyo's data protection laws?
[data_regulation_for_Japan] Q: What are the rights of individuals under Tokyo's data protection laws?
[data_regulation_for_Japan] Q: What are the penalties for non-compliance with data protection laws in Tokyo?
[data_regulation_for_Japan] Q: Are there any specific regulations for cross-border data transfers in Tokyo's data protection laws?
[data_regulation_for_Japan] A: I'm sorry, but I can't provide the information you're looking for.
[data_regulation_for_Japan] A: I cannot provide specific details about Tokyo's data protection laws based on the given context.
[data_regulation_for_Japan] A: The penalties for non-compliance with data protection laws in Tokyo can include fines, administrative sanctions, and potential legal action. Specific penalties may vary

In [ ]:
location = "Shenzhen"
query_engine_tool = get_data_regulation_info(location)
response = query_engine_tool.query_engine.query(f'What are the common elements of the data protection laws for {location}?').response
print(response)

Data regulation law URL not found for this location.


AttributeError: 'NoneType' object has no attribute 'query_engine'